## Извлечение данных из веб-страниц

Модуль requests позволяет получать доступ к веб-страницам. 

Запрос типа get - это когда вы передаете серверу какую-то информацию в адресной строке. Например, если вы перейдете по такому адресу: [https://www.google.ru/?q=python+анализ+данных](https://www.google.ru/?q=python+анализ+данных), то этим вы просите гугл искать по запросу "python анализ данных". 

post-запрос - это когда вам нужно ввести информацию в какую-нибудь форму, например, ввести логин-пароль, который не будет отображать в адресной строке браузера.


In [ ]:
import requests

---
В качестве примера будем использовать данные о биржевых опционах компании Apple Inc., доступных на сайте Yahoo!Finance  

---

In [ ]:
req = requests.get('https://finance.yahoo.com/quote/AAPL/options/')

---
Чтобы проверить, что страница нормально загрузилась используется метод **ok**

---

In [ ]:
req.ok 

In [ ]:
print(req.text)

### Beautiful Soup


Для обработки веб-страниц существует множество пакетов. Проблема с HTML в том, что в вебе много HTML-страниц, написанных не по стандарту HTML. Впрочем, обработка даже не вполне корректного HTML-кода не так сложна, если под рукой есть подходящие инструменты.

Пакет **Beautiful Soup 4** входит в стандартную поставку Anaconda, но если вы используете другой дистрибутив Python, возможно, вам придётся его установить вручную с помощью pip install beautifulsoup4.

In [ ]:
from bs4 import BeautifulSoup

---
Чтобы использовать Beautiful Soup, нужно передать функции BeautifulSoup текст веб-страницы (в виде одной строки). 

html.parser - парсер (программа, которая осуществляет обработку HTML), входит в поставку Python и не требует установки

---


In [ ]:
page = BeautifulSoup(req.text, 'html.parser')

In [ ]:
page

---
Объект **page** очень похож на строку, на самом деле, это не просто строка. К page можно делать запросы. Например:

---

In [ ]:
page.html

---
 Можно пойти вглубь и посмотреть на содержимое `<head>`

---

In [ ]:
page.html.head

---
Теперь мы видим только то, что внутри тега `<head>`. Мы можем пойти еще глубже, и получить то, что находится внутри тега `<title>`, который в свою очередь находится внутри тега `<head>` (говорят, что `<title>` является *потомком* `<head>`:

---

In [ ]:
page.html.head.title

In [ ]:
page.title

---
У тегов, кроме названия, бывают еще свойства — например, в строчке `<html lang="en-US">` мы видим свойство `lang` у тега `<html>`, имеющее значение `"en-US"`. 

---

In [ ]:
page.html['lang']

---
Другим важным примером тега со свойствами является тег `<a>`, который создает ссылку. У него есть свойство `href`, которое хранит собственно ссылку.


---

In [ ]:
page('a')

In [ ]:
for link in page('a'):
    print(link['href'])

---
Одним из потомков `<body>` является `<table>`. Ее можно получить вот так.

---

In [ ]:
page.body.table

---
Допустим, что нам необходимо получить несколько элементов с одинаковым тегом, например, все строки `<tr>`. Для этого может быть использован такой синтаксис:

---

In [ ]:
rows = page.body.table.findAll('tr')
rows

In [ ]:
len(rows)

---
Мы видим, что список не пуст. Так что по нему можно пройти циклом.

---

In [ ]:
for i, row in enumerate(rows):
    print(i)
    print(row)

---
У нас есть строчки и каждая из них является таким же объектом BeautifulSoup, как и все предыдущие. Так что к ним можно применить конструкцию row.td

---

In [ ]:
for i, row in enumerate(rows):
    print(i)
    print(row.td)

---
Мы видим, что если внутри тега `<row>` есть несколько тегов `<td>`, то row.td возьмет первый из них. Поэтому мы получили первый столбец. Но нас интересует не сам тег `<td>`, а строка, которая там лежит. Её можно напечатать вот так.

---

In [ ]:
del rows[0] # предварительно удалим первый элемент, т.к. он пуст
for i, row in enumerate(rows):
    print(i)
    print(row.td.string)

---
Давайте загрузим таблицу в виде списка списков

---

In [ ]:
table = []
for i, row in enumerate(rows):
    table.append([])
    for cell in row.findAll('td'):
        table[-1].append(cell.string)
print(table)

---
Можно написать короче:

---

In [ ]:
table = [[cell.string.strip() for cell in row('td')] 
         for row in rows]
print(table)

In [ ]:
import pandas as pd
dt = pd.DataFrame(table)
dt

### Поиск данных на HTML-странице


Заметим, что в Википедии встречаются ссылки двух типов: внутренние (на другие страницы Википедии) и внешние (на другие сайты), причём они различаются по оформлению — у внешних ссылок есть небольшая стрелочка. Например, мы хотим выбрать все внешние ссылки. Как это сделать?


Для того, чтобы браузер отображал внешние ссылки не так, как внутренние, разработчики Википедии используют так называемые css-классы (конечно, это касается не только Википедии — это вообще основной инструмент современного веба). Теги `<a>`, соответствующие внешним ссылкам, имеют специальный атрибут `class`, значение которого включает слово `external`. Именно по нему можно понять, что речь идёт о внешней ссылке. Это можно было бы увидеть, изучив исходный код страницы, но можно сделать проще: воспользоваться встроенным в браузер инспектором кода.


В исходном коде в атрибуте class тега `<a>` указана строчка "external text", а не просто "external" — дело в том, что теги могут иметь сразу несколько классов одновременно, и в данном случае external и text — это два класса данной ссылки. Но мы будем ориентироваться только на external.

Итак, мы хотим найти все ссылки с классом external. Это очень просто.

In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
url = "https://ru.wikipedia.org/w/index.php?oldid=75475510"

In [ ]:
req = requests.get(url)

In [ ]:
req.ok

In [ ]:
page = BeautifulSoup(req.text, "html.parser")

In [ ]:
for link in page.findAll("a", class_='external'):
    print(link['href'])

Как видно из примера выше, достаточно методу `findAll()` передать дополнительный именованный параметр `class_` — обратите внимание на нижнее подчёркивание, без него получится синтаксическая ошибка, потому что слово `class` имеет специальный смысл в Python.

### Классы и поиск по дереву
Решим теперь другую задачу: допустим, мы хотим найти все ссылки в разделе «Примечания», где находятся сноски к основному тексту. С помощью инспектора кода в Firefox (или аналогичных инструментов для других браузеров) мы легко можем заметить, что весь этот раздел находится внутри тега `<div>` (этот тег описывает прямоугольные блоки, из которых состоят веб-страницы, и является основным тегом для современной веб-вёрстки), имеющем класс `reflist columns reflist-narrow`.

In [ ]:
divs = page.findAll('div', class_='reflist columns reflist-narrow')

In [ ]:
len(divs)

---
Такой `<div>` оказался единственным на странице. Найдём теперь все теги `<a>`, являющиеся потомками (возможно, отдалёнными) этого `<div>`'а.

---

In [ ]:
div = page.findAll('div', class_='reflist columns reflist-narrow')[0]
for link in div("a")[0:20]:
    print(link['href'])

---
Для экономии места вывеведены только первые 20 ссылок. Это внутренние ссылки на другие фрагменты страницы, поэтому они начинаются с символа `#`.

---

**Подведём некоторые итоги по поводу поиска информации в HTML-файлах:**

- Это всегда творческий процесс: все сайты разные и нет единого рецепта, как извлекать из них нужную информацию.
- В первую очередь нужно посмотреть в исходник интересующей вас странички. Проще всего это делать с помощью инструментария веб-разработчика типа Firebug или встроенного инспектора кода в Firefox или аналогичных инструментов для других браузеров.
- В HTML-дереве можно ориентироваться по названиям тегов, их классам, id'ам и другим свойствам.
- Можно искать нужный элемент итеративно — сначала найти «большой» тег, который включает наш элемент, потом найти в нём элемент поменьше и т.д.

### API и XML

Анализируя веб-страницы и извлекая из них информацию мы пытаемся написать программу, которая бы действовала как человек. Это бывает непросто. К счастью, всё чаще разнообразные сайты предлагают информацию, которую может легко обрабатывать не только человек, но и другая программа. Это называется API — application program interface. Обычный интерфейс — это способ взаимодействия человека с программой, а API — одной программы с другой. Например, вашего скрипта на Python с удалённым веб-сервером.

Для хранения веб-страниц, которые читают люди, используется язык HTML. Для хранения произвольных структурированных данных, которыми обмениваются между собой программы, используются другие языки — в частности, язык XML, похожий на HTML. Вернее было бы сказать, что XML это метаязык, то есть способ описания языков. В отличие от HTML, набор тегов в XML-документе может быть произвольным (и определяется разработчиком конкретного диалекта XML). Например, если бы мы хотели описать в виде XML некоторую студенческую группу, это могло бы выглядеть так:

In [ ]:
group = """<group>
<number>134</number>
<student>
<firstname>Виталий</firstname>
<lastname>Иванов</lastname>
</student>
<student>
<firstname>Мария</firstname>
<lastname>Петрова</lastname>
</student>
</group>"""

In [ ]:
obj = BeautifulSoup(group, features="xml")
print(obj.prettify())

Вот так мы можем найти в нашем XML-документе номер группы:

In [ ]:
obj.group.number.string

Это значит «в объекте `obj` найти тег `group` в нём найти тег `number` и выдать в виде строки то, что в нём содержится.

А вот так можно перечислить всех студентов:

In [ ]:
for student in obj.group.findAll('student'):
    print(student.lastname.string, student.firstname.string)



**Задача:** получить списоквсех статей из некоторой категории в Википедии

Вот такой запрос мы можем отправить:

https://en.wikipedia.org/w/api.php?action=query&list=categorymembers&cmtitle=Category:Physics&cmsort=timestamp&cmdir=desc&format=xmlfm

Строка `https://en.wikipedia.org/w/api.php` (до знака вопроса) — это *точка входа* в API. Всё, что идёт после знака вопроса — это, собственно, запрос. Он представляет собой что-то вроде словаря и состоит из пар «ключ=значение», разделяемых амперсандом `&`. Некоторые символы приходится кодировать специальным образом.

Например, в адресе выше сказано, что мы хотим сделать запрос (`action=query`), перечислить элементы категории `list=categorymembers`, в качестве категории, которая нас интересует, указана `Category:Physics` (`cmtitle=Category:Physics`) и указаны некоторые другие параметры. Если кликнуть по этой ссылке, откроется примерно такая штука:

```xml
<?xml version="1.0"?>
<api batchcomplete="">
  <continue cmcontinue="2015-05-30 19:37:50|1653925" continue="-||" />
  <query>
    <categorymembers>
      <cm pageid="24293838" ns="0" title="Wigner rotation" />
      <cm pageid="48583145" ns="0" title="Northwest Nuclear Consortium" />
      <cm pageid="48407923" ns="0" title="Hume Feldman" />
      <cm pageid="48249441" ns="0" title="Phase Stretch Transform" />
      <cm pageid="47723069" ns="0" title="Epicatalysis" />
      <cm pageid="2237966" ns="14" title="Category:Surface science" />
      <cm pageid="2143601" ns="14" title="Category:Interaction" />
      <cm pageid="10844347" ns="14" title="Category:Physical systems" />
      <cm pageid="18726608" ns="14" title="Category:Physical quantities" />
      <cm pageid="22688097" ns="0" title="Branches of physics" />
    </categorymembers>
  </query>
</api>
```



Мы видим здесь разные теги, и видим, что нас интересуют теги `<cm>`, находящиеся внутри тега `<categorymembers>`.

Давайте сделаем соответствующий запрос с помощью Python. Для этого нам понадобится уже знакомый модуль `requests`.

In [ ]:
url = "https://en.wikipedia.org/w/api.php"
params = {
    'action':'query',
    'list':'categorymembers',
    'cmtitle': 'Category:Physics',
    'format': 'xml'
}

req = requests.get(url, params=params)
req.ok

In [ ]:
data = BeautifulSoup(req.text, features='xml')

In [ ]:
print(data.prettify())

Найдём все вхождения тега `<cm>` и выведем их атрибут `title`:

In [ ]:
for cm in data.api.query.categorymembers("cm"):
    print(cm['title'])

Можно было упростить поиск `<cm>`, не указывая «полный путь» к ним:

In [ ]:
for cm in data("cm"):
    print(cm['title'])

По умолчанию сервер вернул нам список из 10 элементов. Если мы хотим больше, нужно воспользоваться элементом continue — это своего рода гиперссылка на следующие 10 элементов.

In [ ]:
data.find("continue")['cmcontinue']

Необходимо использовать метод find() вместо того, чтобы просто написать data.continue, потому что continue в Python имеет специальный смысл.

Теперь добавим cmcontinue в наш запрос и выполним его ещё раз:

In [ ]:
params['cmcontinue'] = data.api("continue")[0]['cmcontinue']

In [ ]:
req = requests.get(url, params=params)
data = BeautifulSoup(g.text, features='xml')
for cm in data.api.query.categorymembers("cm"):
    print(cm['title'])

Мы получили следующие 10 элементов из категории. Продолжая таким образом, можно выкачать её даже целиком (правда, для этого потребуется много времени).

Аналогичным образом реализована работа с разнообразными другими API, имеющимися на разных сайтах. Где-то API является полностью открытым (как в Википедии), где-то вам потребуется зарегистрироваться и получить application id и какой-нибудь ключ для доступа к API, где-то попросят даже заплатить (например, автоматический поиск в Google). Есть API, которые позволяют только читать информацию, а бывают и такие, которые позволяют её править. Например, можно написать скрипт, который будет автоматически сохранять какую-то информацию в Google Spreadsheets. Всякий раз при использовании API вам придётся изучить его документацию, но это в любом случае проще, чем обрабатывать HTML-код. Иногда удаётся упростить доступ к API, используя специальные библиотеки.

## Задание

1.	Необходимо получить данные (название, ссылка, стоимость) с https://market.yandex.ru/ о какой-нибудь группе товаров. Например: https://market.yandex.ru/search?text=Конструкторы%20LEGO&cvredirect=0&track=redirbarup&local-offers-first=0
2.	Представить собранные данные в виде DataFrame, записать в файл.
3.	Кратко пояснить выбор инструментов сбора данных (HTML, API (xml/json)). 

P.S.: Можно выбрать другой ресурс для сбора данных и/или изменить список полей итогового DataFrame по своему усмотрению. Аргументировать/согласовать выбор источника/набор данных.
